# TDD in High Gear and Low Gear

* In the previous chapter, we added a service layer, which helps clearly define our use cases and workflows.
* Still, most of our unit tests are operating on the model.
* Is there an advantage to moving these tests up to the service layer?

## How Is Our Test Pyramid Looking?

* If we organise our tests in folders: *tests/unit/*, *tests/integration/*, *tests/e2e*, we can use `grep -c test_ test_*.py` to count how many of each we have.
* If we `num_unit > num_integration > num_e2e`, our test pyramid is looking good.

## Should Domain Layer Tests Move To The Service Layer?

* Let's look at the same test in the domain layer and in the service layer.

In [2]:
# domain-layer test
def test_prefers_current_stock_batches_to_shipments():
    in_stock_batch = model.Batch("in-stock-batch", "chair", 100, eta=date.today())
    shipment_batch = model.Batch("shipment-batch", "chair", 100, eta=date.tomorrow())
    line = model.OrderLine("oref", "chair", 10)
    
    model.allocate(line, [in_stock_batch, shipment_batch])
    
    assert in_stock_batch.available_quantity == 90
    assert shipment_batch.available_quantity == 100
    
# service-layer test
def test_prefers_warehouse_batches_to_shipments():
    in_stock_batch = model.Batch("in-stock-batch", "chair", 100, eta=date.today())
    shipment_batch = model.Batch("shipment-batch", "chair", 100, eta=date.tomorrow())
    line = model.OrderLine("oref", "chair", 10)
    
    repo = FakeRepository([in_stock_batch, shipment_batch])
    session = FakeSession()
    
    services.allocate(line, repo, session)
    
    assert in_stock_batch.available_quantity == 90
    assert shipment_batch.available_quantity == 100    

* Recall that the domain model is the part of our code most likely to change quickly.
* Writing many unit tests against something so fluid can discourage teams from making necessary changes.
* Testing against the service layer rather than the domain model means that we can still check the domain model's behaviour without relying on its implementation.

## On Deciding What Kind Of Tests To Write

* Should you then refactor away all domain model tests?
* When we find that code is difficult to use or has a code smell, you should refactor.
* However, tests at a high level, e.g., e2e tests, tell us nothing about the fine-grained design of our objects.
* On the other hand, we can change the entire implementation of our domain. If our high-level test still passes, it gives us confidence that a large-scale change hasn't broken the code.
* It often makes sense to start by using tests to sketch out the design at a low level. At some point, though, we will need to replace or delete these tests because they are tightly coupled to implementation.

## High and Low Gear

* When adding a new feature or fixing a bug, we don't need to make big changes to the domain model.
* In such cases, we prefer to write tests against services because of lower coupling and higher coverage.
* When starting a new project or hitting a difficult problem, drop back down to writing tests against to domain model so that you get more direct feedback.
* The analogy is akin to shifting gears on a bicycle. At the start, we begin in low gear to overcome inertia. Once we've built some momentum up, we shift into a high gear. If we encounter a steep climb, we should shift back into low gear.

## Fully Decoupling Service Layer Tests From The Domain

* Our service layer tests still explicitly depend on the domain not changing, as in using `model.OrderLine` and `model.Batch`.
    * If the implementation of either of these changes, we'll need to refactor all affected tests
* Instead we can use fixtures that work with primitives.

In [4]:
class FakeRepository:
    
    @staticmethod
    def for_batch(ref, sku, quantity, eta):
        return FakeRepository([model.Batch(ref, sku, quantity, eta)])

* Now, the dependency only happens in one place, so we only need to fix a single line.

### Adding A Missing Service

* Oftentimes, however, if we find ourselves adding domain-layer stuff into our service-layer tests, this just means that our service layer is incomplete.
    * For example, instead of the above modification of `FakeRepository`, we can create a new service-layer function `add_batch`

In [5]:
def add_batch(ref, sku, quantity, eta, repo, session):
    repo.add(model.Batch(ref, sku, quantity, eta))
    session.commit()

* We can test using this function to create batches and add them to the repository.
* **Note:** You should not write a new service layer function just to remove a dependency. If you're going to need such a function anyway though, you can use it to clean up your tests.

## Wrap Up

* Rules of Thumb:
    * Aim for one E2E test per feature, which demonstrates the feature actually works as a whole
    * Write the bulk of your tests against the service layer
    * Maintain a small core of tests against your domain model
    * Error-handling counts as a feature
    * Express your service layer in terms of primitives rather than in terms of domain objects